In [30]:
import pandas as pd
from os import walk,path
from os.path import join
from nltk import word_tokenize

file_text = {}

for root,curr_dir,files in walk('./_sources'):
    for file in files:
        with open(join(root,file),'r',errors='ignore') as f:
            if path.splitext(file)[1] == '.rst' and \
                'unit' in root.strip().lower() and \
                'topic' in file.strip().lower():
                file_text[path.splitext(file)[0].strip().lower()] = f.read()

In [31]:
def process_read_count(rst_text):
    rst_text = rst_text.strip().lower()
    line_count = 0
    rst_text_lines = rst_text.splitlines()
    act_list = ['mchoice','hparsons','activecode','parsonsprob','fillintheblank','shortanswer','youtube']

    for line in rst_text_lines:
        match_count = 0
        for token in word_tokenize(line.strip().lower()):
            if token in act_list:
                match_count += 1
        
        ## if match_count remains zero
        if match_count == 0:
            line_count += 1
    # if there are lines to read in the page the count the page for reading
    return 1 if line_count > 0 else 0


def process_video_count(rst_text):
    rst_text = rst_text.strip().lower()
    video_count = 0
    rst_text_lines = rst_text.splitlines()
    for line in rst_text_lines:
        if 'youtube' in line.strip().lower():
            video_count += 1
    return video_count

def process_act_count(rst_text):
    rst_text = rst_text.strip().lower()
    rst_text_lines = rst_text.splitlines()
    act_list = ['mchoice','hparsons','activecode','parsonsprob','fillintheblank','shortanswer']
    act_count = 0
    for line in rst_text_lines:
        for token in word_tokenize(line.strip().lower()):
            if token in act_list:
                act_count += 1

    return act_count
    

In [32]:
df = pd.DataFrame.from_dict({"Level..Chapter.":list(file_text.keys()),
                             "page_count":list([process_read_count(x) for x in file_text.values()]),
                             "video_count":list([process_video_count(x) for x in file_text.values()]),
                             "act_count":list([process_act_count(x) for x in file_text.values()])
                             })
df = df.sort_values('Level..Chapter.')
df.loc[:,'page_counter'] = df['page_count'].cumsum()
df.loc[:,'video_counter'] = df['video_count'].cumsum()
df.loc[:,'act_counter'] = df['act_count'].cumsum()

df.to_csv('./csawesome_page_act_video_counts_per_page.csv',index=False)